<a href="https://colab.research.google.com/github/jojoconverteo/Evaneos/blob/main/Evaneos_Churn_(clustering).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![](https://www.offremedia.com/sites/default/files/vignette/article/converteo-logo.png)

# **Cours classification partie 3 : Analyse non supervisée**


-------------------------------
Contexte : On cherche à savoir si un client mobile est à risque de désengagement


-------------------------------

**Dictionnaire :**

Churn (Cible) :
-  1 if customer cancelled service, 0 if not

AccountWeeks : 
- number of weeks customer has had active account


DataPlan : 
- 1 if customer has data plan, 0 if not

DataUsage : 
 - gigabytes of monthly data usage


CustServCalls : 
- number of calls into customer service


DayMins :
- average daytime minutes per month


DayCalls : 
- average number of daytime calls


MonthlyCharge :
- average monthly bill


OverageFee :
- largest overage fee in last 12 months

ContractRenewal :
- 1 if customer recently renewed contract, 0 if not


RoamMins : 
- average number of roaming minutes [LE ROAMING, C'EST QUOI ?](https://www.sfrbusiness.fr/room/communications-unifiees/roaming-c-est-quoi.html)

In [ ]:
#@title


!pip install plotly --upgrade
!pip install -U pandas_profiling

from pandas_profiling import ProfileReport
from google.colab import drive
import os 
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import typing
from typing import List
import numpy as np


import warnings
warnings.filterwarnings('ignore')

def func_create_noise(df_train_data: pd.DataFrame, coloumns_cat_2_category: List) -> pd.DataFrame:
  """
  Fonction qui permet de creer du bruit


  Parameters:
  ----------------------------
    df_train_data: pd.DataFrame 
    Dataframe d'entree 


  Return:
  -----------------------------
    df_train_data_suffle: pd.DataFrame
    Dataframe modifier  

  """
  dict_create_noise_columns_cat = {1 : 'Yes', 0 : 'No'}
  df_train_data_suffle = df_train_data.sample(frac=1)

  for col in columns_cat_2_category: 
    df_train_data_suffle[col] = df_train_data_suffle[col].apply(lambda x: dict_create_noise_columns_cat[x])

  return df_train_data_suffle




drive.mount('/content/drive')
sep = os.sep
str_path_to_file = f"/content/drive/My Drive/Cours Data/Classification/Data/telecom_churn.csv"

df_train_data_suffle = pd.read_csv(str_path_to_file, encoding='ascii')

columns_cat_2_category=['Churn', 'ContractRenewal', 'DataPlan']
df_train_data_suffle = func_create_noise(df_train_data_suffle, columns_cat_2_category)
df_train_data_suffle = df_train_data_suffle.sample(frac=1)

df_train_data_suffle_cat = df_train_data_suffle.select_dtypes(include=object)
df_train_data_suffle_num = df_train_data_suffle.select_dtypes(exclude=object)

-----------------
 
### K-means
 







In [ ]:
pd.get_dummies(df_train_data_suffle_cat)

,Churn_No,Churn_Yes,ContractRenewal_No,ContractRenewal_Yes,DataPlan_No,DataPlan_Yes
3270,1,0,0,1,1,0
2193,1,0,0,1,1,0
2628,1,0,0,1,1,0
3289,1,0,0,1,1,0
319,0,1,1,0,0,1
...,...,...,...,...,...,...
2166,1,0,0,1,1,0
156,0,1,0,1,1,0
2999,1,0,0,1,1,0
834,1,0,0,1,1,0


-------------------------------------
One-hot-encoding amelioré

In [ ]:
#@title

df_train_data_one_hot_encoding = pd.get_dummies(df_train_data_suffle_cat, drop_first=True)
df_train_data_one_hot_encoding

,Churn_Yes,ContractRenewal_Yes,DataPlan_Yes
3270,0,1,0
2193,0,1,0
2628,0,1,0
3289,0,1,0
319,1,0,1
...,...,...,...
2166,0,1,0
156,1,1,0
2999,0,1,0
834,0,1,0


In [ ]:
#@title

df_train_data_kmeans = pd.concat([df_train_data_suffle_num, df_train_data_one_hot_encoding], axis=1)
df_train_data_kmeans

,AccountWeeks,DataUsage,CustServCalls,DayMins,DayCalls,MonthlyCharge,OverageFee,RoamMins,Churn_Yes,ContractRenewal_Yes,DataPlan_Yes
3270,60,0.00,2,180.3,67,49.0,10.40,12.8,0,1,0
2193,123,0.26,2,257.9,92,64.6,10.58,9.4,0,1,0
2628,37,0.30,4,221.0,126,58.0,10.23,6.8,0,1,0
3289,39,0.00,1,187.2,110,42.0,5.74,13.2,0,1,0
319,128,2.54,2,223.5,81,80.4,9.44,9.4,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...
2166,166,0.26,2,197.9,89,57.6,12.55,11.2,0,1,0
156,83,0.00,0,337.4,120,77.0,11.37,15.8,1,1,0
2999,76,0.00,1,204.0,69,54.0,11.26,9.6,0,1,0
834,104,0.00,3,76.4,116,23.0,5.78,9.4,0,1,0


In [ ]:
#@title

from sklearn.preprocessing import StandardScaler

df_train_data_kmeans = pd.concat([df_train_data_suffle_num, df_train_data_one_hot_encoding], axis=1)
sk_scaler = StandardScaler()
sk_scaler.fit(df_train_data_kmeans)
array_train_data_kmeans = sk_scaler.transform(df_train_data_kmeans)

from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
 
 
inertia = []
for n_clusters in range(2, 14):
   kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(array_train_data_kmeans)
   inertia.append(kmeans.inertia_ / n_clusters)
 
inertias = pd.DataFrame({'n_clusters': range(2, 14), 'inertia': inertia})
 
fig = px.line(inertias, x="n_clusters", y="inertia", title='Inertie (somme des variances des clusters) par nombre de clusters')
fig.show()



In [ ]:
#@title
kmeans = KMeans(n_clusters=3, random_state=0).fit(array_train_data_kmeans)

df_train_data_kmeans['Clusteur'] = kmeans.labels_

In [ ]:
#@title

import plotly.graph_objects as go

list_col_num = [w for w in df_train_data_suffle_num.columns]
list_col_little = ['DataUsage', 'CustServCalls', 'OverageFee', 'RoamMins']
list_col_num_big = [w for w in list_col_num if w not in list_col_little]
list_col_cat = [w for w in df_train_data_kmeans.columns if w not in list_col_num and w != 'Clusteur']

from typing import List

def return_viz_kmeans_compare(df_train_data_kmeans: pd.DataFrame, list_:List, *args) -> None:
  fig = go.Figure(data=[
      go.Bar(name='Clusteur_0', x=list_, y=df_train_data_kmeans.query('Clusteur == 0')[list_].mean().to_list()),
      go.Bar(name='Clusteur_1', x=list_, y=df_train_data_kmeans.query('Clusteur == 1')[list_].mean().to_list()),
      go.Bar(name='Clusteur_2', x=list_, y=df_train_data_kmeans.query('Clusteur == 2')[list_].mean().to_list())
  ])

  fig.update_layout(barmode='group')
  fig.show()



list_clusteur = ['Clusteur_0', 'Clusteur_1', 'Clusteur_3']
return_viz_kmeans_compare(df_train_data_kmeans , list_col_num_big)
return_viz_kmeans_compare(df_train_data_kmeans , list_col_little)
return_viz_kmeans_compare(df_train_data_kmeans , list_col_cat)